In [42]:
import numpy as np

def cholesky_decomposition(A):
    """Выполняет разложение Холецкого для матрицы A."""
    n = A.shape[0]
    L = np.zeros_like(A)
    
    for i in range(n):
        for j in range(i + 1):
            sum_k = np.dot(L[i, :j], L[j, :j])
            if i == j:
                L[i, j] = np.sqrt(A[i, i] - sum_k)
            else:
                L[i, j] = (A[i, j] - sum_k) / L[j, j]
    return L

def solve_cholesky(L, b):
    """Решает систему уравнений, используя разложение Холецкого."""
    n = L.shape[0]
    y = np.zeros(n)
    
    # Прямой ход (решение Ly = b)
    for i in range(n):
        y[i] = (b[i] - np.dot(L[i, :i], y[:i])) / L[i, i]
    
    # Обратный ход (решение L^T x = y)
    x = np.zeros(n)
    for i in reversed(range(n)):
        x[i] = (y[i] - np.dot(L.T[i, i+1:], x[i+1:])) / L.T[i, i]
    
    return x

# Пример использования
A = np.array([
    [0.83, 2.18, -1.73],
    [2.18, -1.41, 1.03],
    [-1.73, 1.03, 2.27]
], dtype=float)

b = np.array([0.28, -1.18, 0.72], dtype=float)

# Проверка симметричности матрицы
if not np.allclose(A, A.T):
    raise ValueError("Матрица A не симметрична")

try:
    L = cholesky_decomposition(A)
    print("Матрица L после разложения Холецкого:\n", L)
    
    x = solve_cholesky(L, b)
    print("\nРешение системы:\n", x)
    
except np.linalg.LinAlgError as e:
    print("Ошибка: матрица не положительно определена")

Матрица L после разложения Холецкого:
 [[ 0.91104336  0.          0.        ]
 [ 2.39286087         nan  0.        ]
 [-1.8989217          nan         nan]]

Решение системы:
 [nan nan nan]


C:\Users\Данил\AppData\Local\Temp\ipykernel_3484\1815697337.py:12: RuntimeWarning: invalid value encountered in sqrt
  L[i, j] = np.sqrt(A[i, i] - sum_k)


In [14]:
import numpy as np

def simple_iteration_with_steps(B, c, epsilon=0.001, max_iter=1000):
    n = len(c)
    x_prev = np.zeros(n)
    history = []
    
    # 1. Проверка необходимого и достаточного условий
    B_norm = np.max(np.sum(np.abs(B), axis=0))  # 1-норма матрицы
    spectral_radius = np.max(np.abs(np.linalg.eigvals(B)))  # Спектральный радиус
    
    print("Теоретические проверки:")
    print(f"1. Достаточное условие: ||B||_1 = {B_norm:.2f} {'< 1' if B_norm < 1 else '≥ 1'}")
    print(f"2. Необходимое условие: ρ(B) = {spectral_radius:.2f} {'< 1' if spectral_radius < 1 else '≥ 1'}")
    
    if B_norm >= 1 and spectral_radius >= 1:
        raise ValueError("Метод гарантированно расходится: оба условия нарушены")
    elif spectral_radius >= 1:
        print("ρ(B) ≥ 1. Метод может расходиться!")
    elif B_norm >= 1:
        print("||B||₁ ≥ 1. Сходимость не гарантируется, но возможна")
    else:
        print("Оба условия выполнены. Метод гарантированно сходится")
    
    print("-" * 40)
    
    # 2. Итерационный процесс
    for iter in range(max_iter):
        x_current = B @ x_prev + c
        diff = x_current - x_prev
        norm = np.max(np.abs(diff))
        
        history.append({
            "iteration": iter + 1,
            "x": x_current.copy(),
            "norm": norm,
            "spectral_radius": spectral_radius
        })
        
        print(f"Итерация {iter + 1}:")
        print(f"x = {np.round(x_current, 4)}")
        print(f"Δx: {norm:.4f}")
        print("-" * 40)
        
        if norm < epsilon:
            break
            
        x_prev = x_current.copy()
    else:
        print(f"Точность не достигнута за {max_iter} итераций")
    
    return history

# ----------------------------------------------------------
B = np.array([
    [0.00, 0.22, -0.11, 0.31],
    [0.38, 0.00, -0.12, 0.22],
    [0.11, 0.23, 0.00, -0.51],
    [0.17, -0.21, 0.31, 0.00]
], dtype=float)

c = np.array([2.7, -1.5, 1.2, -0.17], dtype=float)

history = simple_iteration_with_steps(B, c, epsilon=0.001)

# Результат
final_solution = history[-1]["x"]
print("\nФинальное решение (x₁-x₄):")
for i, x in enumerate(final_solution.round(3), 1):
    print(f"x{i} = {x:.3f}")

Теоретические проверки:
1. Достаточное условие: ||B||_1 = 1.04 ≥ 1
2. Необходимое условие: ρ(B) = 0.46 < 1
||B||₁ ≥ 1. Сходимость не гарантируется, но возможна
----------------------------------------
Итерация 1:
x = [ 2.7  -1.5   1.2  -0.17]
Δx: 2.7000
----------------------------------------
Итерация 2:
x = [ 2.1853 -0.6554  1.2387  0.976 ]
Δx: 1.1460
----------------------------------------
Итерация 3:
x = [ 2.7221 -0.6035  0.7919  0.7231]
Δx: 0.5368
----------------------------------------
Итерация 4:
x = [ 2.7043 -0.4015  0.9918  0.665 ]
Δx: 0.2020
----------------------------------------
Итерация 5:
x = [ 2.7087 -0.4451  1.066   0.6815]
Δx: 0.0742
----------------------------------------
Итерация 6:
x = [ 2.6961 -0.4487  1.048   0.7144]
Δx: 0.0329
----------------------------------------
Итерация 7:
x = [ 2.7075 -0.4441  1.029   0.7074]
Δx: 0.0190
----------------------------------------
Итерация 8:
x = [ 2.7084 -0.439   1.0349  0.7025]
Δx: 0.0059
--------------------------------

In [40]:
import numpy as np

def seidel_method_v2(A, b, epsilon=0.001, max_iter=100):
    """
    Второй вариант метода Зейделя:
    - Используется разложение A = D + U + L, где U — верхняя треугольная (без диагонали)
    - Формула итерации: x^{(k+1)} = (D + U)^{-1} (b - L x^{(k)})
    """
    n = len(b)
    x = np.zeros(n)
    
    # Построение матриц D, U, L
    D = np.diag(np.diag(A))
    U = np.triu(A, 1)  # Верхняя треугольная без диагонали
    L = np.tril(A, -1)  # Нижняя треугольная без диагонали
    
    try:
        # Матрица итераций F = -(D + U)^(-1) * L
        F = -np.linalg.inv(D + U) @ L
    except np.linalg.LinAlgError:
        raise ValueError("Матрица (D+U) вырождена")
    
    # Проверка условий сходимости
    F_norm = np.max(np.sum(np.abs(F), axis=0))  # Норма ||F||₁
    spectral_radius = np.max(np.abs(np.linalg.eigvals(F)))
    
    print("Теоретические проверки:")
    print(f"1. Достаточное условие (||F||₁ < 1): {F_norm:.4f} {'✓' if F_norm < 1 else '✗'}")
    print(f"2. Необходимое условие (ρ(F) < 1): {spectral_radius:.4f} {'✓' if spectral_radius < 1 else '✗'}")
    
    if F_norm >= 1 and spectral_radius >= 1:
        print("Оба условия нарушены")
    elif F_norm >= 1:
        print("Сходимость не гарантируется (||F||₁ ≥ 1)")
    elif spectral_radius >= 1:
        print("Сходимость не гарантируется (ρ(F) ≥ 1)")
    else:
        print("Оба условия выполнены")
    print("=" * 50)
    
    # Итерационный процесс с обратным порядком обновления
    for k in range(max_iter):
        x_old = x.copy()
        
        # Обновление переменных в обратном порядке (x_n, x_{n-1}, ..., x_1)
        for i in reversed(range(n)):
            sum_L = np.dot(A[i,:i], x_old[:i])  # Используем старые значения для нижних элементов
            sum_U = np.dot(A[i,i+1:], x[i+1:])  # Используем обновленные значения для верхних
            x[i] = (b[i] - sum_L - sum_U) / A[i,i]
        
        delta = np.max(np.abs(x - x_old))
        print(f"Итерация {k+1}:")
        print(f"x = {np.round(x, 4)}")
        print(f"Δx: {delta:.4f}")
        print("-" * 50)
        
        if delta < epsilon:
            break
    
    return np.round(x, 3)

# Пример использования
A = np.array([
    [5.6,  2.7, -1.7],
    [3.4, -3.6, -6.7],
    [0.8,  1.3,  3.7]
], dtype=float)

b = np.array([1.9, -2.4, 1.2], dtype=float)

solution = seidel_method_v2(A, b, epsilon=0.001)

print("\nФинальное решение:")
for i, val in enumerate(solution, 1):
    print(f"x{i} = {val:.3f}")

Теоретические проверки:
1. Достаточное условие (||F||₁ < 1): 2.2781 ✗
2. Необходимое условие (ρ(F) < 1): 0.3174 ✓
Сходимость не гарантируется (||F||₁ ≥ 1)
Итерация 1:
x = [0.4073 0.0631 0.3243]
Δx: 0.4073
--------------------------------------------------
Итерация 2:
x = [0.0895 0.6529 0.2141]
Δx: 0.5899
--------------------------------------------------
Итерация 3:
x = [0.0679 0.6105 0.0756]
Δx: 0.1385
--------------------------------------------------
Итерация 4:
x = [0.1012 0.5537 0.0951]
Δx: 0.0568
--------------------------------------------------
Итерация 5:
x = [0.1013 0.5614 0.1079]
Δx: 0.0128
--------------------------------------------------
Итерация 6:
x = [0.098  0.5667 0.1051]
Δx: 0.0053
--------------------------------------------------
Итерация 7:
x = [0.0982 0.5656 0.104 ]
Δx: 0.0011
--------------------------------------------------
Итерация 8:
x = [0.0985 0.5651 0.1044]
Δx: 0.0005
--------------------------------------------------

Финальное решение:
x1 = 0.098
x2 = 0